In [1]:
# ==========================
# 📌 CELL 1 — Imports & Config
# ==========================

import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt

CONFIG = {
    "SEED": 42,
    "IMG_SIZE": 256,
    "BATCH_SIZE": 128,         # will be split across both GPUs
    "NUM_WORKERS": 16,
    "LR": 1e-4,
    "EPOCHS": 10,
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu"
}

torch.manual_seed(CONFIG["SEED"])
np.random.seed(CONFIG["SEED"])


In [2]:
# ====================================
# 📌 CELL 2 — Parse DEFACTO dataset (with cache)
# ====================================

import os
import pandas as pd
from glob import glob
from tqdm import tqdm

BASE_PATH = "/media/behzadhassan/datasets/Defacto/"

DATASET_PATHS = {
    'splicing': BASE_PATH + 'defacto-splicing/',
    'copymove': BASE_PATH + 'defacto-copymove/',
    'inpainting': BASE_PATH + 'defacto-inpainting/',
    'face': BASE_PATH + 'defacto-face/'
}

LABEL_MAP = {
    'splicing': 0,
    'copymove': 1,
    'inpainting': 2,
    'face': 3
}

CACHE_PATH = "defacto_df.pkl"


# ---------- ORIGINAL SCANNER (only used if cache missing) ----------
def collect_defacto():
    rows = []

    print("\n🔍 Scanning full DEFACTO dataset...\n")

    for dtype, root in DATASET_PATHS.items():

        print(f"Processing {dtype.upper()}...")

        img_dirs = sorted(glob(os.path.join(root, "*_img")))

        for img_dir in tqdm(img_dirs):
            image_folder = os.path.join(img_dir, "img")
            images = sorted(glob(os.path.join(image_folder, "*.*")))

            annot_dir = img_dir.replace("_img", "_annotations")
            mask_folder = os.path.join(annot_dir, "probe_mask")

            for img_path in images:
                filename = os.path.basename(img_path).split('.')[0]
                if filename.endswith(".jpg"):
                    filename = filename.replace(".jpg", "")

                mask_matches = glob(os.path.join(mask_folder, f"{filename}.*"))
                if not mask_matches:
                    continue

                rows.append({
                    "image_path": img_path,
                    "mask_path": mask_matches[0],
                    "label": LABEL_MAP[dtype],
                    "type": dtype
                })

    df = pd.DataFrame(rows)
    print("\n====================================")
    print(f"🔥 TOTAL SAMPLES COLLECTED: {len(df)}")
    print("====================================")

    return df



# ---------- CACHE LOADING ----------
if os.path.exists(CACHE_PATH):
    print("⚡ Loading dataset from cache:", CACHE_PATH)
    df = pd.read_pickle(CACHE_PATH)
    print("Loaded:", df.shape)
else:
    print("🚀 Cache not found — scanning now...")
    df = collect_defacto()
    df.to_pickle(CACHE_PATH)
    print(f"🔥 Cache saved to {CACHE_PATH}")

df.head()

⚡ Loading dataset from cache: defacto_df.pkl
Loaded: (229187, 4)


,image_path,mask_path,label,type
0,/media/behzadhassan/datasets/Defacto/defacto-s...,/media/behzadhassan/datasets/Defacto/defacto-s...,0,splicing
1,/media/behzadhassan/datasets/Defacto/defacto-s...,/media/behzadhassan/datasets/Defacto/defacto-s...,0,splicing
2,/media/behzadhassan/datasets/Defacto/defacto-s...,/media/behzadhassan/datasets/Defacto/defacto-s...,0,splicing
3,/media/behzadhassan/datasets/Defacto/defacto-s...,/media/behzadhassan/datasets/Defacto/defacto-s...,0,splicing
4,/media/behzadhassan/datasets/Defacto/defacto-s...,/media/behzadhassan/datasets/Defacto/defacto-s...,0,splicing


In [3]:
# ===========================================
# 📌 CELL 3 — Dataset Class + Transformations
# ===========================================

class DeFactoDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['image_path']
        label = row['label']
        
        # Load image
        image = cv2.imread(img_path)
        if image is None:
            import tifffile
            image = tifffile.imread(img_path)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply transforms
        if self.transforms:
            image = self.transforms(image=image)["image"]

        return image, torch.tensor(label, dtype=torch.long)

def get_transforms(img_size):
    return A.Compose([
        A.RandomResizedCrop(
            size=(img_size, img_size),   # required for Albumentations 2.0.8
            scale=(0.7, 1.0),
            ratio=(0.75, 1.33),
            p=1.0
        ),

        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.2),
        A.RandomBrightnessContrast(p=0.4),
        A.GaussianBlur(blur_limit=(3, 5), p=0.3),
        A.GaussNoise(p=0.3),

        # ⭐ Safe replacement for MaskDropout, Cutout, CoarseDropout
        A.GridDropout(
            ratio=0.5,
            holes_number_x=4,
            holes_number_y=4,
            p=0.3
        ),

        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225)
        ),

        ToTensorV2()
    ])


In [4]:
# =====================================
# 📌 CELL 4 — Split + Balanced Sampler
# =====================================

from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.1, stratify=df["label"], random_state=42)

def create_balanced_sampler(df):
    class_counts = df['label'].value_counts().sort_index().values
    class_weights = 1.0 / class_counts
    sample_weights = df['label'].map(lambda x: class_weights[x]).values
    return WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_sampler = create_balanced_sampler(train_df)

train_dataset = DeFactoDataset(train_df, transforms=get_transforms(CONFIG["IMG_SIZE"]))
val_dataset   = DeFactoDataset(val_df,   transforms=get_transforms(CONFIG["IMG_SIZE"]))

train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG["BATCH_SIZE"],
    sampler=train_sampler,
    num_workers=CONFIG["NUM_WORKERS"],
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG["BATCH_SIZE"],
    shuffle=False,
    num_workers=CONFIG["NUM_WORKERS"],
    pin_memory=True
)

/tmp/ipykernel_2962230/2726931275.py:48: UserWarning: Argument(s) 'holes_number_x, holes_number_y' are not valid for transform GridDropout
  A.GridDropout(
/home/behzadhassan/anaconda3/envs/dl/lib/python3.10/site-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
# =====================================
# 📌 CELL 5 — Load ResNet50 + DataParallel
# =====================================

from torchvision import models
import torch.nn as nn
import torch.optim as optim
import torch
from collections import OrderedDict

def create_resnet50(num_classes=4):
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(2048, num_classes)
    return model

# Create model architecture
model = create_resnet50(num_classes=4)
model = model.to(CONFIG["DEVICE"])

# ---- LOAD CHECKPOINT ----
checkpoint_path = "best_resnet50_defacto.pth"
state = torch.load(checkpoint_path, map_location=CONFIG["DEVICE"])

# If checkpoint was saved from DataParallel, remove "module." prefixes
new_state = OrderedDict()
for k, v in state.items():
    new_state[k.replace("module.", "")] = v

model.load_state_dict(new_state)

print("✅ Loaded previous ResNet50 checkpoint successfully!")

# ---- MULTI-GPU ----
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs with DataParallel")
    model = nn.DataParallel(model)

# ---- Loss & Optimizer ----
criterion = nn.CrossEntropyLoss()

# 🔥 Recommended for fine-tuning: lower LR
optimizer = optim.Adam(model.parameters(), lr=CONFIG["LR"] * 0.1)
print("⚡ Using LR =", CONFIG["LR"] * 0.1)

✅ Loaded previous ResNet50 checkpoint successfully!
Using 2 GPUs with DataParallel
⚡ Using LR = 1e-05


In [7]:
# =====================================
# 📌 CELL 6 — Continue Training (10 Epochs)
# =====================================

from torch.cuda.amp import GradScaler, autocast

EXTRA_EPOCHS = 10
scaler = GradScaler()
best_val_acc = 0.0

for epoch in range(EXTRA_EPOCHS):
    model.train()
    train_correct = 0
    train_total = 0

    pbar = tqdm(train_loader, desc=f"Extra Epoch {epoch+1}/{EXTRA_EPOCHS}")

    for imgs, labels in pbar:
        imgs = imgs.to(CONFIG["DEVICE"])
        labels = labels.to(CONFIG["DEVICE"])

        optimizer.zero_grad()

        with autocast():
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        preds = outputs.argmax(dim=1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

        pbar.set_postfix({
            "loss": loss.item(),
            "acc": train_correct/train_total
        })

    # ---- Validation ----
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(CONFIG["DEVICE"])
            labels = labels.to(CONFIG["DEVICE"])

            with autocast():
                outputs = model(imgs)

            preds = outputs.argmax(dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    print(f"Validation Accuracy: {val_acc:.4f}")

    # Save if improved
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_resnet50_defacto.pth")
        print("🔥 Saved improved ResNet50:", best_val_acc)

/tmp/ipykernel_1770858/1606758852.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/home/behzadhassan/anaconda3/envs/dl/lib/python3.10/site-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/tmp/ipykernel_1770858/1606758852.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Extra Epoch 1/10: 100%|█| 1612/1612 [07:49<00:00,  3.43it/s, loss=0.315, acc=0.8
/tmp/ipykernel_1770858/1606758852.py:51: FutureWarning: `torch.

Validation Accuracy: 0.8905
🔥 Saved improved ResNet50: 0.8905275099262621


Extra Epoch 2/10: 100%|█| 1612/1612 [07:32<00:00,  3.56it/s, loss=0.276, acc=0.8


Validation Accuracy: 0.8905


Extra Epoch 3/10: 100%|█| 1612/1612 [07:39<00:00,  3.51it/s, loss=0.271, acc=0.8


...

Validation Accuracy: 0.9027
🔥 Saved improved ResNet50: 0.9027444478380383


Extra Epoch 9/10: 100%|█| 1612/1612 [07:32<00:00,  3.56it/s, loss=0.307, acc=0.9


Validation Accuracy: 0.8965


Extra Epoch 10/10: 100%|█| 1612/1612 [07:32<00:00,  3.56it/s, loss=0.236, acc=0.


Validation Accuracy: 0.8996


In [8]:
torch.save(model.state_dict(), "best_resnet50_defacto2.pth")

In [8]:
# =====================================
# 📌 CELL — Predict Class from Image
# =====================================

import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# ---- CLASS MAPPING ----
CLASS_NAMES = ["splicing", "copymove", "inpainting", "face"]

# ---- LOAD MODEL ----
def load_resnet_model(weights_path="best_resnet50_defacto.pth"):
    model = models.resnet50(weights=None)
    model.fc = nn.Linear(2048, 4)

    # Load checkpoint
    state = torch.load(weights_path, map_location="cpu")

    # Remove DataParallel prefixes if present
    new_state = {}
    for k, v in state.items():
        new_state[k.replace("module.", "")] = v

    model.load_state_dict(new_state)
    model.eval()
    return model

model = load_resnet_model()

# ---- PREPROCESS ----
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    )
])

# ---- PREDICT FUNCTION ----
def predict_class(image_path):
    img = Image.open(image_path).convert("RGB")
    tensor = transform(img).unsqueeze(0)  # [1,3,H,W]

    with torch.no_grad():
        output = model(tensor)
        pred_idx = output.argmax(dim=1).item()
        pred_class = CLASS_NAMES[pred_idx]

    return pred_class

print("✅ Prediction cell ready to use!")


✅ Prediction cell ready to use!


In [17]:
predict_class("/home/behzadhassan/DL/Defacto/1/1.tif")

'copymove'

In [5]:
# ====================================================
# 📌 Evaluate Classification Model (Accuracy, Precision, Recall, F1)
# ====================================================
from torchvision import models
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from tqdm import tqdm

# -----------------------------------------
# Load trained classification model
# -----------------------------------------
def load_resnet_model(weights_path="best_resnet50_defacto.pth"):
    model = models.resnet50(weights=None)
    model.fc = nn.Linear(2048, 4)

    # Load weights
    state = torch.load(weights_path, map_location="cpu")

    # Remove "module." from DataParallel
    clean_state = {}
    for k, v in state.items():
        clean_state[k.replace("module.", "")] = v

    model.load_state_dict(clean_state)
    return model


model = load_resnet_model()
model = model.to(CONFIG["DEVICE"])
model.eval()

# -----------------------------------------
# Evaluate on validation set
# -----------------------------------------
all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in tqdm(val_loader, desc="Evaluating"):
        imgs = imgs.to(CONFIG["DEVICE"])
        labels = labels.to(CONFIG["DEVICE"])

        outputs = model(imgs)
        preds = outputs.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# -----------------------------------------
# Compute metrics
# -----------------------------------------
accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(
    all_labels, all_preds, average="weighted"
)

print("\n===============================")
print(" Classification Metrics")
print("===============================")
print(f"Accuracy :  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")
print("===============================\n")

# Optional — per class report
print(classification_report(all_labels, all_preds, target_names=["splicing", "copymove", "inpainting", "face"]))


/home/behzadhassan/anaconda3/envs/dl/lib/python3.10/site-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Evaluating: 100%|█████████████████████████████| 180/180 [00:47<00:00,  3.79it/s]



 Classification Metrics
Accuracy :  0.9027
Precision: 0.9091
Recall   : 0.9027
F1-score : 0.9053

              precision    recall  f1-score   support

    splicing       0.96      0.91      0.93     10639
    copymove       0.56      0.60      0.58      1820
  inpainting       0.68      0.77      0.72      2500
        face       1.00      1.00      1.00      7960

    accuracy                           0.90     22919
   macro avg       0.80      0.82      0.81     22919
weighted avg       0.91      0.90      0.91     22919

